## Laboratorio 07 - Investigación Profunda

¡Uno de los casos de uso clásicos de Agentic en diferentes empresas! ¡Esto es enorme!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#00bfff;">Implicaciones comerciales</h2>
            <span style="color:#00bfff;">Un agente de Investigación Profunda es ampliamente aplicable a cualquier área de negocio y a tus actividades diarias. ¡Puedes usarlo tú mismo!
            </span>
        </td>
    </tr>
</table>

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [3]:
load_dotenv(override=True)

True

## Herramientas alojadas de OpenAI

El SDK de agentes de OpenAI incluye las siguientes herramientas alojadas:

- La herramienta `WebSearchTool` permite a un agente buscar en la web.
- La herramienta `FileSearchTool` permite recuperar información de sus almacenes de vectores de OpenAI.
- La herramienta `ComputerTool` permite automatizar tareas informáticas como tomar capturas de pantalla y hacer clic.

### Nota importante: Costo de la API de la herramienta `WebSearchTool`

Esto me cuesta 2,5 centavos por llamada para la herramienta `WebSearchTool` de OpenAI. Esto puede suponer entre 2 y 3 dólares para los próximos dos laboratorios. Usaremos herramientas de búsqueda de bajo coste con otras plataformas, así que no dude en omitir esta opción si le preocupa el coste.

Los costes están disponibles aquí: https://platform.openai.com/docs/pricing#web-search

In [4]:
INSTRUCTIONS = "Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y \
producí una descripción concisa de los resultados. La descripción debe tener 2-3 párrafos y menos de 300 \
palabras. Captura los puntos principales. Escribe de manera concisa, no es necesario tener frases completas o buena \
gramática. Esto será consumido por alguien que está sintetizando un informe, por lo que es vital que captures el \
esencia y ignores cualquier fluff. No incluyas ningún comentario adicional más que la descripción en sí."

search_agent = Agent(
    name="Agente de búsqueda",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Últimos frameworks de agentes de IA en 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

En 2025, emergieron varios frameworks de agentes de inteligencia artificial (IA) que han transformado el desarrollo y la implementación de sistemas autónomos. Google presentó Antigravity, un entorno de desarrollo integrado (IDE) que permite a los desarrolladores delegar tareas complejas a agentes de IA autónomos, impulsados principalmente por el modelo Gemini 3 Pro. Antigravity ofrece una arquitectura centrada en agentes, facilitando flujos de trabajo de codificación asíncronos y verificables. ([es.wikipedia.org](https://es.wikipedia.org/wiki/Google_Antigravity?utm_source=openai))

Por otro lado, LightAgent se destacó como un framework de código abierto diseñado para la creación de agentes de IA autónomos. Integrando funcionalidades clave como memoria, herramientas y árboles de pensamiento, LightAgent mantiene una estructura ligera y se integra fácilmente con plataformas de chat populares, permitiendo a los desarrolladores construir agentes autoaprendices de manera eficiente. ([arxiv.org](https://arxiv.org/abs/2509.09292?utm_source=openai))

Además, DMAS-Forge emergió como un framework que facilita el despliegue transparente de aplicaciones de IA como sistemas distribuidos. Este framework desacopla la lógica de la aplicación de las decisiones específicas de despliegue, generando automáticamente el código y las configuraciones necesarias para implementar aplicaciones multiagente distribuidas en diversos escenarios, reduciendo significativamente el esfuerzo manual. ([arxiv.org](https://arxiv.org/abs/2510.11872?utm_source=openai))

Estos avances reflejan una tendencia hacia la creación de frameworks que priorizan la autonomía, la eficiencia y la integración fluida de agentes de IA en diversos entornos de desarrollo. 

### Como siempre, revisamos la traza:

https://platform.openai.com/traces

### Ahora utilizaremos salidas estructuradas e incluiremos una descripción de los campos

In [6]:
# Consulte la nota anterior sobre el costo de WebSearchTool

HOW_MANY_SEARCHES = 6

INSTRUCTIONS = f"Eres un asistente de investigación útil. Dado un término de búsqueda, \
produce un conjunto de búsquedas web para realizar para responder la consulta. \
Salida: {HOW_MANY_SEARCHES} términos para consultar."

# Usamos Pydantic para definir el esquema de nuestra respuesta; esto se conoce como "Salidas Estructuradas".

class WebSearchItem(BaseModel):
    reason: str = Field(description="Tu razonamiento de por qué esta búsqueda es importante para la consulta.")

    query: str = Field(description="El término de búsqueda para usar para la búsqueda web.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="Una lista de búsquedas web a realizar para responder la consulta.")


planner_agent = Agent(
    name="Agente de planificación",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [7]:
message = "Últimos frameworks de agentes de IA en 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Investigar los frameworks de IA más recientes y avanzados para entender su aplicación en 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='Buscar artículos y reseñas que analicen las tendencias y desarrollos en frameworks de inteligencia artificial para el año 2025.', query='AI frameworks trends 2025'), WebSearchItem(reason='Explorar las mejores prácticas y casos de uso de frameworks de agentes de IA disponibles en 2025.', query='best practices AI agent frameworks 2025')]


In [8]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto y el cuerpo HTML proporcionados """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("edward@passportai.app") # Cambiar a tu correo electrónico verificado
    to_email = To("ofazzito@yahoo.com.ar") # Cambiar a tu correo electrónico
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [9]:
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el asunto y el cuerpo HTML proporcionados', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C7BD99A0C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [10]:
INSTRUCTIONS = """Eres capaz de enviar un correo electrónico HTML bien formateado basado en un informe detallado.
Se te proporcionará un informe detallado. Debes usar tu herramienta para enviar un correo electrónico, proporcionando el 
informe convertido en HTML limpio, bien presentado con un asunto adecuado."""

email_agent = Agent(
    name="Agente de correo electrónico",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [12]:
INSTRUCTIONS = (
    "Eres un investigador senior encargado de escribir un informe coherente para una consulta de investigación. "
    "Se te proporcionará la consulta original y algunas investigaciones iniciales realizadas por un asistente de investigación.\n"
    "Primero, debes elaborar un esquema para el informe que describa la estructura y "
    "flujo del informe. Luego, genera el informe y devuelve ese como tu salida final.\n"
    "La salida final debe estar en formato markdown, y debe ser larga y detallada "
    "para 5-10 páginas de contenido, al menos 1000 palabras."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="Un resumen de 2-3 párrafos de los resultados.")

    markdown_report: str = Field(description="El informe final")

    follow_up_questions: list[str] = Field(description="Temas sugeridos para investigar más")


writer_agent = Agent(
    name="Agente de escritura",
    instructions=INSTRUCTIONS,
    model="gpt-4o",
    output_type=ReportData,
)

### Las siguientes 3 funciones planificarán y ejecutarán la búsqueda, utilizando planner_agent y search_agent

In [13]:
async def plan_searches(query: str):
    """ Utilice planner_agent para planificar qué búsquedas ejecutar para la consulta """
    print("Planificando búsquedas...")
    result = await Runner.run(planner_agent, f"Consulta: {query}")
    print(f"Se realizarán {len(result.final_output.searches)} búsquedas")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Llama a search() para cada elemento en el plan de búsqueda """
    print("Buscando...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Búsqueda finalizada")
    return results

async def search(item: WebSearchItem):
    """ Usa el agente de búsqueda para ejecutar una búsqueda web para cada elemento en el plan de búsqueda """
    input = f"Término de búsqueda: {item.query}\nRazón para buscar: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### Las siguientes 2 funciones escriben un informe y lo envían por correo electrónico.

In [14]:
async def write_report(query: str, search_results: list[str]):
    """ Usa el agente de escritura para escribir un informe basado en los resultados de la búsqueda """
    print("Pensando sobre el informe...")
    input = f"Consulta original: {query}\nResultados de búsqueda resumidos: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Informe finalizado")
    return result.final_output

async def send_email(report: ReportData):
    """ Usa el agente de correo electrónico para enviar un correo electrónico con el informe """
    print("Escribiendo correo electrónico...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Correo electrónico enviado")
    return report

### ¡Hora del espectáculo!

In [15]:
query = "Últimos frameworks de agentes de IA en 2025"

with trace("Investigación"):
    print("Iniciando investigación...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("¡Felicidades!")




Iniciando investigación...
Planificando búsquedas...
Se realizarán 3 búsquedas
Buscando...
Búsqueda finalizada
Pensando sobre el informe...
Informe finalizado
Escribiendo correo electrónico...
Correo electrónico enviado
¡Felicidades!


### Como siempre, revisa la traza del resultado:

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#00cc00;">Felicitaciones por tu progreso y una solicitud</h2>
<span style="color:#00cc00;">Has llegado a un momento importante con el curso; has creado un agente valioso usando uno de los frameworks de agentes más recientes. Has mejorado tus habilidades y has descubierto nuevas posibilidades comerciales. ¡Tómate un momento para celebrar tu éxito! Y piensa como podriamos implemetar este agente de investigacion en módulos de python<br/>
        </td>
    </tr>